#### Data can be downloaded [here](https://digital.wpi.edu/concern/student_works/5712m8080?locale=en)
Just scroll to the bottom of the page, download the .zip file, and unzip it! The sheet you're looking for is 'Accuscore Evaluation.xlsx'

I chose to 

In [ ]:
# import packages and dataset
import pandas as pd

QB = pd.read_excel('../Data/Accuscore Evaluation.xlsx', sheet_name='QB Projections')
DST = pd.read_excel('../Data/Accuscore Evaluation.xlsx', sheet_name= 'DST Projections')